In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.cross_validation import train_test_split, StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [2]:
def create_dataset_right(rs_tag, mins_diff=5):
    rf = pd.read_csv('a13/Rechts_Flow_2011.csv')
    rs = pd.read_csv('a13/Rechts_Speed_2011.csv')

    jam_threshold = rs[rs_tag].quantile(.25)

    y = (rs[rs_tag] < jam_threshold)[mins_diff:]

    df = pd.concat([rf,rs],axis=1)

    df = df.iloc[:-mins_diff,:]

    return df, y

def train_val_test_split(df, y, test_size=0.15, val_size=0.15):
    cut1 = int(len(y) * test_size)
    cut2 = int(len(y) * (test_size + val_size))

    X_train = df.iloc[:-cut2,:]
    X_val = df.iloc[-cut2:-cut1,:]
    X_test = df.iloc[-cut1:,:]

    y_train = y[:-cut2]
    y_val = y[-cut2:-cut1]
    y_test = y[-cut1:]

    return X_train, X_val, X_test, y_train, y_val, y_test


In [3]:
df, y = create_dataset_right(rs_tag='118.4', mins_diff=5)

In [4]:
train_dataset, valid_dataset, test_dataset, train_labels, valid_labels, test_labels  =  train_val_test_split(df, y, test_size=0.15, val_size=0.15)

In [5]:
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (367916, 286) (367916,)
Validation set (78839, 286) (78839,)
Test set (78839, 286) (78839,)


In [6]:
num_labels = 2

def reformat(dataset, labels):
    dataset = dataset.as_matrix().astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

def labels_decode(labels):
    return [r.tolist().index(1) for r in labels]

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (367916, 286) (367916, 2)
Validation set (78839, 286) (78839, 2)
Test set (78839, 286) (78839, 2)


### Multilayer model with regularization + learning rate decay

In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [8]:
import tensorflow as tf

nfeats = train_dataset.shape[1]
num_nodes1 = 1024
num_nodes2 = 300
num_labels = 2
batch_size = 200

beta1=0.001
beta2=0.001
beta3=0.0001

ratio = sum(y) / len(y)

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, nfeats))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    class_weight = tf.constant([ratio, 1.0 - ratio])

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([nfeats, num_nodes1]))
    biases_1 = tf.Variable(tf.zeros([num_nodes1]))

    weights_2 = tf.Variable(tf.truncated_normal([num_nodes1, num_nodes2]))
    biases_2 = tf.Variable(tf.zeros([num_nodes2]))

    weights_final = tf.Variable(tf.truncated_normal([num_nodes2, num_labels]))
    biases_final= tf.Variable(tf.zeros([num_labels]))


    # Training computation.
    input_relus1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
#     input_relus1 = tf.nn.dropout(input_relus1, 0.2)
    relu_layer1 = tf.nn.relu(input_relus1)
    
    input_relus2 = tf.matmul(relu_layer1, weights_2) + biases_2
#     input_relus2 = tf.nn.dropout(input_relus2, 0.2)
    relu_layer2 = tf.nn.relu(input_relus2)
    
    logits = tf.matmul(relu_layer2, weights_final) + biases_final
    weighted_logits = tf.mul(logits, class_weight)

    loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(weighted_logits, tf_train_labels))
    regularization = beta1 * tf.nn.l2_loss(weights_1) + beta2 * tf.nn.l2_loss(weights_2) +\
                     beta3 * tf.nn.l2_loss(weights_final)
    loss += regularization
    
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
#     learning_rate = tf.train.exponential_decay(0.1, global_step, 1000, 0.90, staircase=True)
#     learning_rate = 0.1
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.5,
                                          initial_accumulator_value=0.1,
                                          use_locking=False,
                                          name='Adagrad').minimize(loss)
    
#     optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.001,
#                                         rho=0.95, epsilon=1e-08,
#                                         use_locking=False, name='Adadelta').minimize(loss)
    
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(tf.matmul(
                                tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),
                                weights_2
                        ) + biases_2),
            weights_final
        ) + biases_final
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(tf.matmul(
                                tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),
                                weights_2
                        ) + biases_2),
            weights_final
        ) + biases_final
    )

In [9]:
num_steps = 500001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                                        valid_prediction.eval(), valid_labels))
    y_true = test_labels
    y_pred = test_prediction.eval()
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 95432.375000
Minibatch accuracy: 0.0%
Validation accuracy: 76.5%
Minibatch loss at step 100: 251.543610
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Minibatch loss at step 200: 234.096405
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 300: 23056.246094
Minibatch accuracy: 94.0%
Validation accuracy: 91.4%
Minibatch loss at step 400: 742690.250000
Minibatch accuracy: 0.0%
Validation accuracy: 77.3%
Minibatch loss at step 500: 28705.501953
Minibatch accuracy: 81.0%
Validation accuracy: 49.3%
Minibatch loss at step 600: 29249.654297
Minibatch accuracy: 91.0%
Validation accuracy: 46.6%
Minibatch loss at step 700: 195082.203125
Minibatch accuracy: 66.5%
Validation accuracy: 70.8%
Minibatch loss at step 800: 15335.548828
Minibatch accuracy: 3.5%
Validation accuracy: 23.4%
Minibatch loss at step 900: 8695.810547
Minibatch accuracy: 1.5%
Validation accuracy: 23.5%
Minibatch loss at step 1000: 189.166183
Minibatch accur

KeyboardInterrupt: 

In [ ]:
y_pred

In [ ]:
def labels_decode(labels):
    return [r.tolist().index(1) for r in labels]

In [ ]:
def logits_decode(logits):
    return [r.argmax() for r in logits]


In [ ]:
y_true = labels_decode(y_true)
y_pred = logits_decode(y_pred)

In [ ]:
print(classification_report(y_true, y_pred))